<a href="https://colab.research.google.com/github/sallihin/hdb-resale-predictor/blob/main/training-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data.gov.sg - Resale Flat Prices\
https://data.gov.sg/dataset/resale-flat-prices


In [3]:
import pandas as pd
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error 

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 


import sklearn.externals
import joblib

In [4]:
df = pd.read_csv("/content/drive/MyDrive/MLDV/Assessment/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")

### Data Exploration

In [5]:
df.shape

(140168, 11)

In [6]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


### Data Preparation


**Feature Selection**
* As HDB flats have 99 years lease, the lease_commence_date and remaining_lease are highly correlated to each other. 
* I will also remove the block numbers and street_name to reduce noise and prevent overfitting.


In [7]:
df = df.drop(columns=['remaining_lease', 'block', 'street_name'])
df.head()

,month,town,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,2017-01,ANG MO KIO,2 ROOM,10 TO 12,44.0,Improved,1979,232000.0
1,2017-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,New Generation,1978,250000.0
2,2017-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,New Generation,1980,262000.0
3,2017-01,ANG MO KIO,3 ROOM,04 TO 06,68.0,New Generation,1980,265000.0
4,2017-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,New Generation,1980,265000.0


**One-Hot Encoding**

In [8]:
features_df = pd.get_dummies(df, columns=['town', 'flat_type', 'storey_range', 'flat_model'])

In [9]:
del features_df['resale_price']
print(features_df.columns)

Index(['month', 'floor_area_sqm', 'lease_commence_date', 'town_ANG MO KIO',
       'town_BEDOK', 'town_BISHAN', 'town_BUKIT BATOK', 'town_BUKIT MERAH',
       'town_BUKIT PANJANG', 'town_BUKIT TIMAH', 'town_CENTRAL AREA',
       'town_CHOA CHU KANG', 'town_CLEMENTI', 'town_GEYLANG', 'town_HOUGANG',
       'town_JURONG EAST', 'town_JURONG WEST', 'town_KALLANG/WHAMPOA',
       'town_MARINE PARADE', 'town_PASIR RIS', 'town_PUNGGOL',
       'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG', 'town_SERANGOON',
       'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS', 'town_YISHUN',
       'flat_type_1 ROOM', 'flat_type_2 ROOM', 'flat_type_3 ROOM',
       'flat_type_4 ROOM', 'flat_type_5 ROOM', 'flat_type_EXECUTIVE',
       'flat_type_MULTI-GENERATION', 'storey_range_01 TO 03',
       'storey_range_04 TO 06', 'storey_range_07 TO 09',
       'storey_range_10 TO 12', 'storey_range_13 TO 15',
       'storey_range_16 TO 18', 'storey_range_19 TO 21',
       'storey_range_22 TO 24', 'storey_ra

In [10]:
features_df.head()

,month,floor_area_sqm,lease_commence_date,town_ANG MO KIO,town_BEDOK,town_BISHAN,town_BUKIT BATOK,town_BUKIT MERAH,town_BUKIT PANJANG,town_BUKIT TIMAH,town_CENTRAL AREA,town_CHOA CHU KANG,town_CLEMENTI,town_GEYLANG,town_HOUGANG,town_JURONG EAST,town_JURONG WEST,town_KALLANG/WHAMPOA,town_MARINE PARADE,town_PASIR RIS,town_PUNGGOL,town_QUEENSTOWN,town_SEMBAWANG,town_SENGKANG,town_SERANGOON,town_TAMPINES,town_TOA PAYOH,town_WOODLANDS,town_YISHUN,flat_type_1 ROOM,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_4 ROOM,flat_type_5 ROOM,flat_type_EXECUTIVE,flat_type_MULTI-GENERATION,storey_range_01 TO 03,storey_range_04 TO 06,storey_range_07 TO 09,storey_range_10 TO 12,storey_range_13 TO 15,storey_range_16 TO 18,storey_range_19 TO 21,storey_range_22 TO 24,storey_range_25 TO 27,storey_range_28 TO 30,storey_range_31 TO 33,storey_range_34 TO 36,storey_range_37 TO 39,storey_range_40 TO 42,storey_range_43 TO 45,storey_range_46 TO 48,storey_range_49 TO 51,flat_model_2-room,flat_model_Adjoined flat,flat_model_Apartment,flat_model_DBSS,flat_model_Improved,flat_model_Improved-Maisonette,flat_model_Maisonette,flat_model_Model A,flat_model_Model A-Maisonette,flat_model_Model A2,flat_model_Multi Generation,flat_model_New Generation,flat_model_Premium Apartment,flat_model_Premium Apartment Loft,flat_model_Premium Maisonette,flat_model_Simplified,flat_model_Standard,flat_model_Terrace,flat_model_Type S1,flat_model_Type S2
0,2017-01,44.0,1979,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2017-01,67.0,1978,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,2017-01,67.0,1980,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,2017-01,68.0,1980,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,2017-01,67.0,1980,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [11]:
features_df[['year','month']] = df.month.str.split("-",expand=True,)

In [12]:
features_df = features_df.astype({'year': 'int32', 'month': 'int32'})

In [13]:
features_df[['year', 'month']].dtypes

year     int32
month    int32
dtype: object

In [14]:
features_df[['year', 'month']].head()

,year,month
0,2017,1
1,2017,1
2,2017,1
3,2017,1
4,2017,1


### Training Split

In [15]:
X = features_df.values
y = df['resale_price'].values

# Split the data set in a training set (70%) and a test set (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [16]:
# Fit regression model
model = ensemble.GradientBoostingRegressor()

param_grid = {
    'n_estimators': [1000, 3000],
    'max_depth': [4, 6],
    'min_samples_leaf': [3, 9, 17],
    'learning_rate': [0.1, 0.01],
    'max_features': [1.0, 0.1],
    'loss': ['squared_error', 'absolute_error', 'huber']
}

In [18]:
# Define the grid search we want to run. Run it with four cpus in parallel.
# gs_cv = GridSearchCV(model, param_grid, n_jobs=-1, verbose=3)
# gs_cv.fit(X_train, y_train)

# The algorithm took too long and almost exceeded Google Colab's 24hour runtime limit :( 
# Have decided to use another method instead for hyperparameter tuning 

In [ ]:
random_search = RandomizedSearchCV(model, param_grid, n_iter=20, refit=True, verbose=3)
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END learning_rate=0.01, loss=absolute_error, max_depth=4, max_features=1.0, min_samples_leaf=3, n_estimators=1000;, score=0.842 total time= 5.5min
[CV 2/5] END learning_rate=0.01, loss=absolute_error, max_depth=4, max_features=1.0, min_samples_leaf=3, n_estimators=1000;, score=0.843 total time= 5.6min
[CV 3/5] END learning_rate=0.01, loss=absolute_error, max_depth=4, max_features=1.0, min_samples_leaf=3, n_estimators=1000;, score=0.844 total time= 5.5min
[CV 4/5] END learning_rate=0.01, loss=absolute_error, max_depth=4, max_features=1.0, min_samples_leaf=3, n_estimators=1000;, score=0.834 total time= 5.6min
[CV 5/5] END learning_rate=0.01, loss=absolute_error, max_depth=4, max_features=1.0, min_samples_leaf=3, n_estimators=1000;, score=0.835 total time= 5.5min
[CV 1/5] END learning_rate=0.01, loss=squared_error, max_depth=4, max_features=1.0, min_samples_leaf=17, n_estimators=1000;, score=0.888 total time= 5.1min
[C

In [ ]:
print(random_search.best_params_)

In [ ]:
# Find the error rate on the training set using the best parameters
mse = mean_absolute_error(y_train, random_search.predict(X_train))
print("Training Set Mean Absolute Error: %.4f" % mse)

In [ ]:
# Find the error rate on the test set
mse = mean_absolute_error(y_test, random_search.predict(X_test))
print("Test Set Mean Absolute Error: %.4f" % mse)

In [ ]:
# Make predictions on validation dataset
predictions = random_search.predict(X_validation)

print('Accuracy:')
print(accuracy_score(Y_validation, predictions))
print()
print('Confusion matrix:')
print(confusion_matrix(Y_validation, predictions)) #Y-axis is Actual, X-axis is Prediction
print()
print('Classification report:')
print(classification_report(Y_validation, predictions))

In [ ]:
# Save the trained model to a file so we can use it other programs 
joblib.dump(random_search, '/content/drive/MyDrive/MLDV/Assessment/trained_hdb_resale_estimator.pkl')

In [ ]:
house_to_predict = [
  1, # 'month', 
  93, #'floor_area_sqm', 
  2019, #'lease_commence_date', 

  # Select 1 Town 
  0, # 'town_ANG MO KIO',
  0, # 'town_BEDOK',
  0, # 'town_BISHAN', 
  0, # 'town_BUKIT BATOK', 
  0, # 'town_BUKIT MERAH',
  0, # 'town_BUKIT PANJANG', 
  0, # 'town_BUKIT TIMAH', 
  0, # 'town_CENTRAL AREA',
  0, # 'town_CHOA CHU KANG', 
  0, # 'town_CLEMENTI', 
  0, # 'town_GEYLANG', 
  0, # 'town_HOUGANG',
  0, # 'town_JURONG EAST', 
  0, # 'town_JURONG WEST', 
  0, # 'town_KALLANG/WHAMPOA',
  0, # 'town_MARINE PARADE', 
  0, # 'town_PASIR RIS', 
  0, # 'town_PUNGGOL',
  0, # 'town_QUEENSTOWN', 
  0, # 'town_SEMBAWANG', 
  0, # 'town_SENGKANG', 
  0, # 'town_SERANGOON',
  1, # 'town_TAMPINES', 
  0, # 'town_TOA PAYOH', 
  0, # 'town_WOODLANDS', 
  0, # 'town_YISHUN',

  # Select 1 Flat Type
  0, # 'flat_type_1 ROOM', 
  0, # 'flat_type_2 ROOM', 
  0, # 'flat_type_3 ROOM',
  0, # 'flat_type_4 ROOM', 
  1, # 'flat_type_5 ROOM', 
  0, # 'flat_type_EXECUTIVE',
  0, # 'flat_type_MULTI-GENERATION', 

  # Select Floor
  0, # 'storey_range_01 TO 03',
  1, # 'storey_range_04 TO 06', 
  0, # 'storey_range_07 TO 09',
  0, # 'storey_range_10 TO 12', 
  0, # 'storey_range_13 TO 15',
  0, # 'storey_range_16 TO 18', 
  0, # 'storey_range_19 TO 21',
  0, # 'storey_range_22 TO 24', 
  0, # 'storey_range_25 TO 27',
  0, # 'storey_range_28 TO 30', 
  0, # 'storey_range_31 TO 33',
  0, # 'storey_range_34 TO 36', 
  0, # 'storey_range_37 TO 39',
  0, # 'storey_range_40 TO 42', 
  0, # 'storey_range_43 TO 45',
  0, # 'storey_range_46 TO 48', 
  0, # 'storey_range_49 TO 51', 

  # Flat Model
  0, # 'flat_model_2-room',
  0, # 'flat_model_Adjoined flat', 
  0, # 'flat_model_Apartment', 
  0, # 'flat_model_DBSS',
  0, # 'flat_model_Improved', 
  0, # 'flat_model_Improved-Maisonette',
  0, # 'flat_model_Maisonette', 
  0, # 'flat_model_Model A',
  0, # 'flat_model_Model A-Maisonette', 
  0, # 'flat_model_Model A2',
  0, # 'flat_model_Multi Generation', 
  0, # 'flat_model_New Generation',
  0, # 'flat_model_Premium Apartment', 
  0, # 'flat_model_Premium Apartment Loft',
  0, # 'flat_model_Premium Maisonette', 
  0, # 'flat_model_Simplified',
  1, # 'flat_model_Standard', 
  0, # 'flat_model_Terrace', 
  0, # 'flat_model_Type S1',
  0, # 'flat_model_Type S2'],

  2025 # year
]

In [ ]:
predicted_value = random_search.predict([house_to_predict])[0]
print("This house has an estimated value of ${:,.2f}".format(predicted_value))